In [1]:
import spark.ufo as ufo
import utils.spark_utils as su
from utils.utils import get_moon_phase
from utils.address_cleaning import clean_address
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

/home/jgrove/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# spark://172.18.0.4:7077
spark = su.spark_session()
#spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# spark = SparkSession.builder \
#     .appName("MyApp") \
#     .master("spark://localhost:7077") \
#     .getOrCreate()


your 131072x1 screen size is bogus. expect trouble
23/06/07 19:05:27 WARN Utils: Your hostname, helion resolves to a loopback address: 127.0.1.1; using 172.21.218.81 instead (on interface eth0)
23/06/07 19:05:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/jgrove/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jgrove/.ivy2/cache
The jars for the packages stored in: /home/jgrove/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-15009b83-d3cf-4052-b928-6acbb23df0eb;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 85ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

In [4]:

#spark.sql("REFRESH TABLE bronze")
# spark.catalog.refreshTable(r"`./spark-warehouse/ufo/bronze`")
#spark.sql("REFRESH TABLE `'/spark-warehouse/ufo/bronze'`")
df = spark.read.format("delta").load(r"../spark-warehouse/ufo/bronze").limit(5)

df.show()

+-------------+--------------+-----+-------+--------+------------------+--------------------+-------+------+
|     DateTime|          City|State|Country|   Shape|          Duration|             Summary| Posted|Images|
+-------------+--------------+-----+-------+--------+------------------+--------------------+-------+------+
|5/31/22 23:28|      Old Fort|   TN|    USA|   Light|       5-6 seconds|Light that appear...|6/22/22|   Yes|
|5/31/22 23:00|    Long Beach|   CA|    USA|Fireball|       1.5 minutes|Large Hovering Fi...|6/22/22|  null|
|5/31/22 22:34|Lake Hopatcong|   NJ|    USA|   Light|                .5|Red light gets su...|6/22/22|  null|
|5/31/22 22:24|Lake Hopatcong|   NJ|    USA|   Light|10:24:47- 10:24:48|Red light gets su...|6/22/22|  null|
|5/31/22 22:00|        Medina|   OH|    USA|    Star|   approx 1 minute|looked like A ver...|6/22/22|  null|
+-------------+--------------+-----+-------+--------+------------------+--------------------+-------+------+



In [5]:
# Register the get_moon_phase() function as a UDF.
get_moon_phase_udf = udf(get_moon_phase)
clean_address_udf = udf(clean_address)


df = (
    df.filter(df.Country == "USA")
    .withColumn("timestamp", to_timestamp("DateTime", "MM/dd/yy HH:mm"))
    .withColumn("temp_address", concat_ws(", ", df.City, df.State, df.Country))
    .withColumn("address", clean_address_udf("temp_address"))
    .withColumn("city", split("address", ",").getItem(0))
    .withColumn("state", split("address", ",").getItem(1))
    .withColumn("country", split("address", ",").getItem(2))
    .withColumn("latitude", round(split("address", ",").getItem(3).cast("double"), 3))
    .withColumn("longitude", round(split("address", ",").getItem(4).cast("double"), 3))
    .withColumn("date", to_date("timestamp"))
    .withColumn("year", year("date"))
    .withColumn("month", month("date"))
    .withColumn("dayOfWeek", date_format("date", "EEEE"))
    .withColumn("week", weekofyear("date"))
    .withColumn("hour", hour("timestamp"))
    .withColumn("moonPhaseAngle", get_moon_phase_udf("date").cast("double"))
    .withColumnRenamed("Images", "images")
    .drop("DateTime", "Posted", "timestamp", "temp_address")
    #.na.drop(subset=["city", "state", "latitude", "longitude"])
    .dropDuplicates()
    .withColumn("id", monotonically_increasing_id())
    .select(
        "id",
        "city",
        "state",
        "country",
        "latitude",
        "longitude",
        "shape",
        "duration",
        "summary",
        "images",
        "date",
        "year",
        "month",
        "dayOfWeek",
        "week",
        "hour",
        "moonPhaseAngle",
    )
)


df.show()

+---+----------------+-----------+--------------+--------+---------+--------+------------------+--------------------+------+----------+----+-----+---------+----+----+--------------+
| id|            city|      state|       country|latitude|longitude|   shape|          duration|             summary|images|      date|year|month|dayOfWeek|week|hour|moonPhaseAngle|
+---+----------------+-----------+--------------+--------+---------+--------+------------------+--------------------+------+----------+----+-----+---------+----+----+--------------+
|  0|      Long Beach| California| United States|  33.769| -118.192|Fireball|       1.5 minutes|Large Hovering Fi...|  null|2022-05-31|2022|    5|  Tuesday|  22|  23|           0.0|
|  1|            None|       Ohio| United States|    41.1|  -81.938|    Star|   approx 1 minute|looked like A ver...|  null|2022-05-31|2022|    5|  Tuesday|  22|  22|           0.0|
|  2|Roxbury Township| New Jersey| United States|    null|     null|   Light|10:24:47- 10:

In [53]:
df.printSchema()

root
 |-- id: long (nullable = false)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- shape: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- images: string (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofweek: string (nullable = true)
 |-- week: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- moonphase: string (nullable = true)



In [15]:
df_location = (df.select("id", "city", "state", "country"))

df_ufo = (df.select("id", "shape", "summary", "duration"))

df_date = (df.select("date", "year", "month", "dayofweek", "week", "hour"))

df_astronomy = (df.select("id", "moonphase"))


df_location.show()
df_ufo.show()
df_date.show()
df_astronomy.show()

+---+--------------------+---------------+---------+
| id|                city|          state|  country|
+---+--------------------+---------------+---------+
|  0|        Ocean Shores|New South Wales|Australia|
|  1|Cayo Costa state ...|             FL|      USA|
|  2|         Southampton|             MA|      USA|
|  3|ST BRUNO DE MONTA...|         Quebec|   Canada|
|  4|             Bedford|             VA|      USA|
|  5|          Scottsdale|             AZ|      USA|
|  6|           St. Louis|             MO|      USA|
|  7|            St louis|             MO|      USA|
|  8|             WICHITA|             KS|      USA|
|  9|           Skowhegan|             ME|      USA|
| 10|                null|             CA|      USA|
| 11|            Humboldt|             TN|      USA|
| 12|              Encino|             CA|      USA|
| 13|           Texarkana|             AR|      USA|
| 14|              Vernon|             TX|      USA|
| 15|            Paradise|             UT|    